In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("psparks/instacart-market-basket-analysis")

print("Path to dataset files:", path)

d:\code\text-to-sql-agent\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 197M/197M [00:18<00:00, 11.1MB/s] 

Extracting files...


Path to dataset files: C:\Users\rk\.cache\kagglehub\datasets\psparks\instacart-market-basket-analysis\versions\1


In [1]:
import os

dataset_path = r"C:\Users\rk\.cache\kagglehub\datasets\psparks\instacart-market-basket-analysis\versions\1"

# List all files
for file in os.listdir(dataset_path):
    print(file)

aisles.csv
departments.csv
orders.csv
order_products__prior.csv
order_products__train.csv
products.csv


In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Read DB config from environment
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME")

# Validate env vars early
missing = [k for k in ["DB_USER", "DB_PASSWORD", "DB_NAME"] if not os.getenv(k)]
if missing:
    raise RuntimeError(f"Missing env vars: {missing}")

# Create SQLAlchemy engine
engine = create_engine(
    f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# Dataset path
dataset_path = r"C:\Users\rk\.cache\kagglehub\datasets\psparks\instacart-market-basket-analysis\versions\1"

files = [
    ('aisles.csv', 'aisles'),
    ('departments.csv', 'departments'),
    ('products.csv', 'products'),
    ('orders.csv', 'orders'),
    ('order_products__prior.csv', 'order_products_prior'),
    ('order_products__train.csv', 'order_products_train'),
]

for csv_file, table_name in files:
    file_path = os.path.join(dataset_path, csv_file)
    print(f"Loading {csv_file} → {table_name}")

    df = pd.read_csv(file_path)

    if 'order_products' in table_name:
        df.to_sql(
            table_name,
            engine,
            if_exists='replace',
            index=False,
            chunksize=50_000,
            method="multi"
        )
    else:
        df.to_sql(
            table_name,
            engine,
            if_exists='replace',
            index=False
        )

    print(f"{len(df):,} rows loaded")

print("\nInstacart data loaded successfully!")